In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from surprise import accuracy
from surprise.model_selection.validation import cross_validate
from surprise.dataset import Dataset
from surprise.reader import Reader
from surprise import SVD, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split, GridSearchCV
import random

## Calling the Saved Dataset

In [2]:
ratings_df = pd.read_csv("data/ratingData.csv", encoding= 'unicode_escape')
ratings_df.head()

,course_id,user_id,rating
0,2001,1001,5
1,2001,1002,3
2,2001,1003,1
3,2001,1004,0
4,2001,1005,2


## Collaborative Filtering

Lower values of RMSE indicate better fit. RMSE is a good measure of how accurately the model predicts the response. It's the most important criterion for fit if the main purpose of the model is prediction.
Based on a rule of thumb, it can be said that RMSE values between **0.2 and 0.5** shows that the model can relatively predict the data accurately.

There is no correct value for MSE. Simply put, the lower the value the better and 0 means the model is perfect.

### Using Surprise and testing with different algorithms (SVD, KnnBasic, KnnBaseline, KnnWithMeans, KnnWithZScore) to find the best one to use based on MAE and RMAE scores. 

In [3]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings_df[['user_id','course_id','rating']], reader)

#split into training and test sets using 80:20 ratio
raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)                

ratio = int(len(raw_ratings)*0.8)
train_raw = raw_ratings[:ratio] 
test_raw = raw_ratings[ratio:] 

#data is set to the training dataset
data.raw_ratings = train_raw       
trainset = data.build_full_trainset() 
testset = data.construct_testset(test_raw)

#create dict for different models 
models=[SVD(), KNNBasic(), KNNBaseline(), KNNWithMeans(), KNNWithZScore()] 
results = {} #to store the scores

#perform cross validation of MAE and RMSE for all models
for model in models:
    #kfold set to 5
    crossval_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)  
    
    #saving and renaming appropraitely
    result = pd.DataFrame.from_dict(crossval_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE', 'fit_time': 'Fit Time', 'test_time': 'Test Time'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result
    
#printing all models results
all_models = pd.DataFrame.from_dict(results)
all_models.T.sort_values(by='RMSE') #models sorted by RMSE

,MAE,RMSE,Fit Time,Test Time
matrix_factorization.SVD,1.086086,1.340326,0.031944,0.038002
knns.KNNBaseline,1.277351,1.542227,0.006609,0.061850
knns.KNNWithZScore,1.275347,1.547736,0.003855,0.022107
knns.KNNWithMeans,1.280928,1.550474,0.003234,0.023411
knns.KNNBasic,1.314801,1.582670,0.002347,0.065577


### Testing with Different Parameters 
### SVD

####  SVD - Normal

In [4]:
#algo
svd = SVD()

#cross validate with kfold set to 5
cross_validate(svd, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

#build train and train train set
trainset = data.build_full_trainset()
svd.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(svd.test(testset))
accuracy.mse(svd.test(testset))
accuracy.mae(svd.test(testset))

print('\nPredict Tests: ')
print(svd.predict(1001, 2001))
print(svd.predict(2001, 1001))
print(svd.predict(1001, 88))
print(svd.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
svd.test(testset)[:2]

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3260  1.3492  1.3257  1.3626  1.3238  1.3375  0.0157  
MAE (testset)     1.0729  1.0971  1.0707  1.0935  1.0682  1.0805  0.0123  
Fit time          0.04    0.03    0.03    0.03    0.03    0.03    0.00    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    

Accuracy on the testset:
RMSE: 1.3025
MSE: 1.6965
MAE:  1.0158

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 2.64   {'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': False}
user: 1001       item: 88         r_ui = None   est = 2.53   {'was_impossible': False}
user: 1001       item: 5          r_ui = None   est = 2.53   {'was_impossible': False}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=2.643450625453006, details={'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=2.5837231448932565, details={'was_impossible': False})]

#### SVD - Using GridSearchCV 

In [5]:
params = {"n_factors": range(10, 100, 20),
         "n_epochs": [5, 10, 20],
         "lr_all": [0.002, 0.005],
         "reg_all": [0.2, 0.5]}

gsSVD = GridSearchCV(SVD, params, measures = ["RMSE", "MAE"], cv = 5, n_jobs = -1)
gsSVD.fit(data)

print(f'\nRMSE Best Parameters: {gsSVD.best_params["rmse"]}')
print(f'RMSE Best Score: {gsSVD.best_score["rmse"]}')
print(f'MAE Best Parameters: {gsSVD.best_params["mae"]}')
print(f'MAE Best Score: {gsSVD.best_score["mae"]}')


RMSE Best Parameters: {'n_factors': 90, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}
RMSE Best Score: 1.480349872111694
MAE Best Parameters: {'n_factors': 90, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}
MAE Best Score: 1.2472740786053553


##### Using the Best Hyperparameters found above: 

In [6]:
finalSVD = SVD(n_factors = 90, n_epochs = 20, lr_all = 0.005, reg_all = 0.2)
predictions = finalSVD.fit(trainset).test(testset)

print('\nUpdated Accuracy: ')
accuracy.rmse(predictions)
accuracy.mse(predictions)
accuracy.mae(predictions)


Updated Accuracy: 
RMSE: 1.2877
MSE: 1.6583
MAE:  1.0722


1.0722138721670214

### KNNBasic

##### KNNBasic - user-based cosine 

In [7]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
params = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between users
}
algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the cosine similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.5155
MSE: 2.2967
MAE:  1.2614

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.13   {'actual_k': 16, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.128238855166933, details={'actual_k': 16, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.2024891034197556, details={'actual_k': 16, 'was_impossible': False})]

#### KNNBasic - item-based cosine 

In [8]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
params = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between users
}
algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the cosine similarity matrix...


ZeroDivisionError: float division

#### KNNBasic - user-based msd 

In [9]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
params = {
    "name": "msd",
    "user_based": True,  # Compute  similarities between users
}
algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.2245
MSE: 1.4993
MAE:  1.0078

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.13   {'actual_k': 16, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.1268647390529196, details={'actual_k': 16, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.6498068886580213, details={'actual_k': 16, 'was_impossible': False})]

#### KNNBasic - item-based msd 

In [10]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
params = {
    "name": "msd",
    "user_based": False,  # Compute  similarities between users
}
algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 0.7368
MSE: 0.5428
MAE:  0.4596

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 3.16   {'actual_k': 40, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=3.1601592498338467, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=2.9662747627442974, details={'actual_k': 40, 'was_impossible': False})]

#### KNNBasic - cosine, user, shrinkage

In [11]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
params = {
    "name": "msd",
    "user_based": True,
    "shrinkage": 0 #no shrinkage
}
algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.2245
MSE: 1.4993
MAE:  1.0078

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.13   {'actual_k': 16, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.1268647390529196, details={'actual_k': 16, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.6498068886580213, details={'actual_k': 16, 'was_impossible': False})]

#### KNNBasic - msd, user, shrinkage

In [12]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
params = {
    "name": "msd",
    "user_based": False,
    "shrinkage": 0 #shrinkage is 0
}

algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 0.7368
MSE: 0.5428
MAE:  0.4596

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 3.16   {'actual_k': 40, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=3.1601592498338467, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=2.9662747627442974, details={'actual_k': 40, 'was_impossible': False})]

#### KNNBasic - Using GridSearchCV 

In [13]:
#doing this with name is msd or cosine causes float or zero division error
params = {'sim_options' : {'method': ['als','sgd'],
                            "n_epochs" : [5, 10, 20],
                            "user_based": [True, False],
                            "min_support": [3, 5, 8],
                          "user_based": [True, False]}}

gsKB = GridSearchCV(KNNBasic, params, measures=['mae', 'rmse'], cv=5, n_jobs=-1)                               
gsKB.fit(data)

print(f'\nRMSE Best Parameters: {gsKB.best_params["rmse"]}')
print(f'RMSE Best Score: {gsKB.best_score["rmse"]}')
print(f'MAE Best Parameters: {gsKB.best_params["mae"]}')
print(f'MAE Best Score: {gsKB.best_score["mae"]}')


RMSE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'min_support': 3}}
RMSE Best Score: 1.093996338468426
MAE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'min_support': 3}}
MAE Best Score: 0.7795566668131725


#### Using the Best Hyperparameters found above: 

In [14]:
finalKB = KNNBasic(method = "als", n_epochs = 5, user_based = False, min_support = 3)
predictions = finalKB.fit(trainset).test(testset)

print('\nUpdated Accuracy: ')
accuracy.rmse(predictions)
accuracy.mse(predictions)
accuracy.mae(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.

Updated Accuracy: 
RMSE: 1.2245
MSE: 1.4993
MAE:  1.0078


1.0077718091917234

### KNNBaseline

#### KNNBaseline - Using GridSearchCV

In [15]:
params = { 'sim_options' : {'method': ['als','sgd'],
                            "n_epochs" : [5, 10, 20],
                            "user_based": [True, False],
                            "reg_u": [10, 12, 15, 18],
                            "reg_i": [8, 10, 12, 15],
                            "learning_rate": [0.00005, 0.05]}}

gsKBL = GridSearchCV(KNNBaseline, params, measures=['mae', 'rmse'], cv=5, n_jobs=-1)                               
gsKBL.fit(data)

print(f'\nRMSE Best Parameters: {gsKBL.best_params["rmse"]}')
print(f'RMSE Best Score: {gsKBL.best_score["rmse"]}')
print(f'MAE Best Parameters: {gsKBL.best_params["mae"]}')
print(f'MAE Best Score: {gsKBL.best_score["mae"]}')


RMSE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'reg_u': 10, 'reg_i': 8, 'learning_rate': 5e-05}}
RMSE Best Score: 1.0833374989138946
MAE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'reg_u': 10, 'reg_i': 8, 'learning_rate': 5e-05}}
MAE Best Score: 0.7939239934143018


#### Using the Best Hyperparameters found above: 

In [16]:
finalKBL = KNNBaseline(method = "als", n_epochs = 5, user_based = False, reg_u = 10, reg_i = 8)
predictions = finalKBL.fit(trainset).test(testset)

print('\nUpdated Accuracy: ')
accuracy.rmse(predictions)
accuracy.mse(predictions)
accuracy.mae(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.

Updated Accuracy: 
RMSE: 1.1982
MSE: 1.4356
MAE:  0.9791


0.9790664252554542

### KNNWithMeans 

#### KNNWithMeans - Normal 

In [17]:
#algo
algo = KNNWithMeans()

#cross validate with kfold set to 5
cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

#build train and train train set
trainset = data.build_full_trainset()
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5663  1.5445  1.5413  1.5295  1.5270  1.5417  0.0140  
MAE (testset)     1.2939  1.2741  1.2742  1.2678  1.2580  1.2736  0.0118  
Fit time          0.01    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.04    0.02    0.02    0.02    0.02    0.02    0.01    
Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.1983
MSE: 1.4359
MAE:  0.9788

Predict Tests: 
user: 1001       item: 2001       r_ui = No

[Prediction(uid=1001, iid=2001, r_ui=5, est=1.0412444732849224, details={'actual_k': 16, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.8904927575397272, details={'actual_k': 16, 'was_impossible': False})]

##### KNNWithMeans - user-based cosine 

In [18]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
parans = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between users
}
algo = KNNWithMeans(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.1983
MSE: 1.4359
MAE:  0.9788

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.04   {'actual_k': 16, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.0412444732849224, details={'actual_k': 16, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.8904927575397272, details={'actual_k': 16, 'was_impossible': False})]

##### KNNWithMeans - item-based cosine 

In [19]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
parans = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.1983
MSE: 1.4359
MAE:  0.9788

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.04   {'actual_k': 16, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.0412444732849224, details={'actual_k': 16, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.8904927575397272, details={'actual_k': 16, 'was_impossible': False})]

##### KNNWithMeans - user-based MSD 

In [20]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based msd similarity
parans = {
    "name": "msd",
    "user_based": True,  # Compute similarities between users
}
algo = KNNWithMeans(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.1983
MSE: 1.4359
MAE:  0.9788

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.04   {'actual_k': 16, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.0412444732849224, details={'actual_k': 16, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.8904927575397272, details={'actual_k': 16, 'was_impossible': False})]

##### KNNWithMeans - item-based MSD 

In [21]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using item-based msd similarity
parans = {
    "name": "msd",
    "user_based": False,  # Compute similarities between items
}
algo = KNNWithMeans(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.1983
MSE: 1.4359
MAE:  0.9788

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.04   {'actual_k': 16, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.0412444732849224, details={'actual_k': 16, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.8904927575397272, details={'actual_k': 16, 'was_impossible': False})]

#### KNNWithMeans - Using GridSearchCV

In [22]:
#doing this with name is msd or cosine causes float or zero division error
params = {'sim_options' : {'method': ['als','sgd'],
                            "n_epochs" : [5, 10, 20],
                            "user_based": [True, False],
                            "min_support": [3, 5, 8],
                          "user_based": [True, False]}}

gsKWM = GridSearchCV(KNNWithMeans, params, measures=['mae', 'rmse'], cv=5, n_jobs=-1)                               
gsKWM.fit(data)

print(f'\nRMSE Best Parameters: {gsKWM.best_params["rmse"]}')
print(f'RMSE Best Score: {gsKWM.best_score["rmse"]}')
print(f'MAE Best Parameters: {gsKWM.best_params["mae"]}')
print(f'MAE Best Score: {gsKWM.best_score["mae"]}')


RMSE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'min_support': 3}}
RMSE Best Score: 1.1170063996975135
MAE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'min_support': 3}}
MAE Best Score: 0.8313519737369262


#### Using the Best Hyperparameters found above: 

In [23]:
finalKWM = KNNWithMeans(method = "als", n_epochs = 5, user_based = False, min_support = 3)
predictions = finalKWM.fit(trainset).test(testset)

print('\nUpdated Accuracy: ')
accuracy.rmse(predictions)
accuracy.mse(predictions)
accuracy.mae(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.

Updated Accuracy: 
RMSE: 1.1983
MSE: 1.4359
MAE:  0.9788


0.9787717003257882

### KNNWithZScore 

#### KNNWithZScore - Using GridSearchCV

In [24]:
params = {'sim_options' : {'method': ['als','sgd'],
                            "n_epochs" : [5, 10, 20],
                            "user_based": [True, False],
                            "reg_u": [10, 12, 15],
                            "reg_i": [8, 10, 12],
                            "learning_rate": [0.0005, 0.05]}}

gsKZS = GridSearchCV(KNNWithZScore, params, measures=['mae', 'rmse'], cv=5, n_jobs=-1)                               
gsKZS.fit(data)

print(f'\nRMSE Best Parameters: {gsKZS.best_params["rmse"]}')
print(f'RMSE Best Score: {gsKZS.best_score["rmse"]}')
print(f'MAE Best Parameters: {gsKZS.best_params["mae"]}')
print(f'MAE Best Score: {gsKZS.best_score["mae"]}')


RMSE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'reg_u': 10, 'reg_i': 8, 'learning_rate': 0.0005}}
RMSE Best Score: 1.1241581577332298
MAE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'reg_u': 10, 'reg_i': 8, 'learning_rate': 0.0005}}
MAE Best Score: 0.8364530119944135


#### Using the Best Hyperparameters found above: 

In [25]:
finalKZS = KNNWithZScore(method = "als", n_epochs = 5, user_based = False, reg_u = 10, reg_i = 8, learning_rate = 0.0005)
predictions = finalKZS.fit(trainset).test(testset)

print('\nUpdated Accuracy: ')
accuracy.rmse(predictions)
accuracy.mse(predictions)
accuracy.mae(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.

Updated Accuracy: 
RMSE: 1.1986
MSE: 1.4365
MAE:  0.9773


0.9773004281323534